In [1]:
from huggingface_hub import login
login()

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [10]:
# Load a sample dataset
from datasets import load_dataset

ds = load_dataset("databricks/databricks-dolly-15k")

print(ds)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})


In [11]:
def process_dataset(sample):
    output = {
        "messages": [
            {"role": "user", "content": sample["instruction"]},
            {"role": "assistant", "content": sample["response"]},
        ]
    }
    return output
ds = ds.map(process_dataset)

In [12]:
sft_config = SFTConfig(
    output_dir="./checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=4, 
    gradient_accumulation_steps=16,  
    learning_rate=5e-5,
    logging_steps=10, 
    logging_strategy="steps",
    save_steps=100,
    use_mps_device=(
        True if device == "mps" else False
    ) # Use MPS for mixed precision training
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer
)


C:\Users\peera\AppData\Local\Temp\ipykernel_21388\3797389999.py:16: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


In [13]:
# Train the model
trainer.train()

# Save the model
trainer.save_model("./checkpoints")

Step,Training Loss
10,2.655500
20,2.361200
30,2.361900
40,2.346400
50,2.294100
60,2.299600
70,2.268200
80,2.247400
90,2.257000
100,2.213200


In [30]:
model = AutoModelForCausalLM.from_pretrained("./checkpoints").to(device)
tokenizer = AutoTokenizer.from_pretrained("./checkpoints")

prompt = "Who was the first woman to win a Nobel Prize?"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

outputs = model.generate(
    **inputs,
    max_length=256,
    num_beams=4,
    early_stopping=True,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
    length_penalty=1.2,
    temperature=1.0,
    do_sample=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
Who was the first woman to win a Nobel Prize?
assistant
Marie Skłodowska-Curie (1867 – 1934) was a Polish-American physicist and chemist who won the Nobel Prize in Physics in1903 for her work on radioactivity. She was the only person to win two Nobel Prizes.

She was born in Warsaw, Poland and studied at the University of Warsaw, where she graduated in 1881. After studying at the Sorbonne in Paris, she returned to Poland and became a professor of chemistry at the Warsaw Institute of Physics. In the 20th century, she became one of the most famous physicists in the world. She is known for her discovery of radium and polonium, as well as the discovery of polonium and radium salts. She also discovered radium itself, which is a radioactive form of the element uranium. Her research on radioactivity led to the development of the atomic bomb, which resulted in the end of World War II and the establishment of the United Nations. Marie Curie is widely regarded as a pioneer in the field of s